In [1]:
#import Collobrative_Filtering.py as CF
import pandas as pd
import numpy as np
import inspect
import os

In [4]:
file_path = inspect.getfile(inspect.currentframe())
file_direction = os.path.dirname(os.path.abspath(file_path))
offertagging = os.path.join(file_direction, './Demo3/RecommandSystem/MJ_OFFER.xlsx')
#usertagging = os.path.join(file_direction, 'MJ_TAG.xlsx')

In [5]:
offertagging = pd.read_excel(offertagging)

In [6]:
offertagging[['OFFER_ID','LABEL_ID']]

,OFFER_ID,LABEL_ID
0,OFF0007,54
1,OFF0007,54
2,OFF0007,54
3,OFF0007,54
4,OFF0007,54
5,OFF0007,54
6,OFF0007,54
7,OFF0007,54
8,OFF0007,54
9,OFF0007,54


In [4]:
usertagging = pd.read_excel(usertagging)

In [5]:
model = CF.COLLOBORATIVE_FILTERING()

In [7]:
rating_table = model.fit(usertagging,offertagging)

Starting likes info
Number of users: 310
Number of models: 214
Sparsity: 4.702%
Ending likes info
Number of users: 310
Number of models: 214
Sparsity: 4.702%
offer_rating pickle Done!
user_sparse pickle Done!
offerlabel_mapping pickle Done!
tag_mapping pickle Done!


In [12]:
import redis
import json

In [13]:
redis_conn = redis.Redis(host='localhost',port=6379,db=0)

In [ ]:
test = model.predict(json.loads(redis_conn.get(1)))

In [7]:
offertag = pd.read_csv('./Demo3/OFFERTAG_MAPPING.csv')

In [8]:
offertag.head()

,LABEL_ID,OFFER_ID,IND
0,54,OFF0007,1
1,233,OFF0007,1
2,213,OFF0007,1
3,56,OFF0007,1
4,63,OFF0007,1


In [9]:
offertag = offertag.pivot_table(index='OFFER_ID',columns='LABEL_ID',values='IND').fillna(0)

In [10]:
offertag.columns.values

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  17,  19,  20,  21,  22,  23,  24,  25,  27,  28,  29,
        31,  32,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  84,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97, 100, 101, 102, 103, 104,
       106, 107, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119,
       120, 121, 122, 123, 124, 125, 126, 128, 129, 131, 132, 133, 134,
       135, 136, 137, 139, 140, 141, 144, 145, 146, 147, 148, 150, 151,
       152, 153, 155, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 186, 187, 189, 192, 193, 194, 195, 196, 197, 198, 199, 200,
       201, 202, 203, 204, 205, 206, 207, 208, 209, 211, 212, 21

In [11]:
test

NameError: name 'test' is not defined

In [ ]:
np.array(test)*214/250

In [ ]:
tagging_table = np.zeros(214)
for i in np.array(test)*214/250:
    tagging_table[i] = 1

In [ ]:
from sklearn.metrics import pairwise

In [ ]:
pairwise.cosine_similarity(tagging_table.reshape(1,-1), offertag.values)

In [ ]:
offer